In [1]:
import os
import shutil
import urllib3
from subprocess import Popen, PIPE, STDOUT
from multiprocessing import Pool

import numpy as np
from matplotlib import pyplot as plt

### 0. Класс с параметрами атома

In [2]:
class Atom:
    def __init__(self, z, lattice_a, pseudo, atomic_mass):
        self.pseudo = pseudo
        self.celldim = Atom._celldim(lattice_a)
        self.z = z
        self.atomic_mass = atomic_mass
        
    def __str__(self):
        return f'Z={self.z}, celldim={self.celldim}, m={self.atomic_mass}, pseudo={self.pseudo}'
        
    def get_pseudo_file(self):
        return self.pseudo
    
    def get_celldim(self):
        return self.celldim
    
    def get_atomic_mass(self):
        return self.atomic_mass
    
    @staticmethod
    def _celldim(x, a=52.9177):
        # a - радиус Бора в пикометрах
        return x / a

In [3]:
def parse_output(output):
    pattern = ['!', 'total', 'energy']
    
    for line in output.splitlines():
        items = line.split()
        if items[:3] == pattern:
            return float(items[4])

### 1. Конфигурация

In [4]:
pseudo_url2atom = {
    'C': 'https://www.quantum-espresso.org/upf_files/C.pbe-n-kjpaw_psl.1.0.0.UPF',
    'Si': 'https://www.quantum-espresso.org/upf_files/Si.pbe-n-kjpaw_psl.1.0.0.UPF',
    'Ge': 'https://www.quantum-espresso.org/upf_files/Ge.pbe-dn-kjpaw_psl.1.0.0.UPF',
    'Sn': 'https://www.quantum-espresso.org/upf_files/Sn.pbe-dn-kjpaw_psl.1.0.0.UPF'
}
pseudo_dir = 'data/pseudo/pbe'
output_dir = 'output'

# "а" параметр решетки в пикометрах
# Взял тут https://www.webelements.com/tin/crystal_structure.html
lattice_a2atom = {
    'C':  246.4,
    'Si': 543.09,
    'Ge': 565.75,
    'Sn': 583.18,
}

# Атомный номер
z2atom = {
    'C':  6,
    'Si': 14,
    'Ge': 32,
    'Sn': 50
}

atomic_mass = {
    'C':  12.011,
    'Si': 28.085,
    'Ge': 72.630,
    'Sn': 118.710
}

In [5]:
os.makedirs(pseudo_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

In [6]:
http = urllib3.PoolManager()
atoms = {}

for atom_key, url_str in pseudo_url2atom.items():
    url = urllib3.util.parse_url(url_str)
    fname = url.path.split('/')[2]
    path = os.path.join(pseudo_dir, fname)
    
    if not os.path.exists(path): 
        with http.request('GET', url_str, preload_content=False) as r, open(path, 'wb') as f:       
            shutil.copyfileobj(r, f)
        
    atoms[atom_key] = Atom(z=z2atom[atom_key],
                           pseudo=fname,
                           lattice_a=lattice_a2atom[atom_key],
                           atomic_mass=atomic_mass[atom_key])

In [7]:
pwx_lattice_config = """&control
    prefix = '{0}',
    pseudo_dir = '{1}',
    outdir = '{2}',
/
&system
! better lattice parametrization
    ibrav = 2,
    celldm(1) = {3},
    ntyp = 1,
    nat = 2,
    ecutwfc = {4},
    ecutrho = {5},
/
&electrons
/
ATOMIC_SPECIES
    X {6} {7}
ATOMIC_POSITIONS alat
    X 0.00 0.00 0.00
    X 0.25 0.25 0.25
K_POINTS automatic
    6 6 6 1 1 1
"""

In [8]:
pwx_exe='pw'
ecutwfc = 47
ecutrho = 250
sigma = 1e-2

### 2. Расчет объемного модуля упругости B

In [9]:
atom = atoms['Ge']
print(atom)

Z=32, celldim=10.691129811008414, m=72.63, pseudo=Ge.pbe-dn-kjpaw_psl.1.0.0.UPF


In [25]:
def variate_celldm(undeformed_celldim):
    compression_celldim = undeformed_celldim - undeformed_celldim * 1e-2
    stretching_celldim = undeformed_celldim + undeformed_celldim * 1e-2
    return (compression_celldim, stretching_celldim)

def numerical_derivative(num, num_low, num_high, delta):
    return (num_low - 2 * num + num_high) / (2 * (delta ** 2))

def calc_gamma(num):
    return ((num * 0.529) ** 3) / 8

def calc_B(gamma, d2E):    
    B = (1 / (9 * gamma)) * d2E * 1.602
    return B

def calc_C11(gamma, d2Ec11, d2Ec12):
    C11 = (1 / (4 * gamma) * (d2Ec11 + d2Ec12))
    return C11

def calc_C12(gamma, d2Ec11, d2Ec12):
    C12 = (1 / (4 * gamma) * (d2Ec11 - d2Ec12))
    return C12

def calc_C44(gamma, d2E):
    C44 = (1 / gamma) * d2E
    return C44

In [11]:
undeformed_celldim = atom.get_celldim()
compression_celldim, stretching_celldim = variate_celldm(undeformed_celldim)

In [12]:
def calc_total_energy(celldim):
    conf = pwx_lattice_config.format(f'lattice_{celldim}',
                                    pseudo_dir,
                                    output_dir,
                                    celldim,
                                    ecutwfc,
                                    ecutrho,
                                    atom.get_atomic_mass(),
                                    atom.get_pseudo_file())
    p = Popen([pwx_exe], stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    stdout, _ = p.communicate(input=conf.encode())
    output = stdout.decode()
    return parse_output(output)

In [13]:
undeformed_celldim_energy = calc_total_energy(atom.get_celldim())
compression_celldim_energy = calc_total_energy(compression_celldim)
stretching_celldim_energy = calc_total_energy(stretching_celldim)

gamma = calc_gamma(undeformed_celldim)
d2E = numerical_derivative(undeformed_celldim_energy, compression_celldim_energy, stretching_celldim_energy, sigma) * 13.6
B = calc_B(gamma, d2E) 

In [30]:
print(f'B = {B}')

B = 0.7509711810475209


### 3. Расчет константы эластичности С44

In [15]:
def matrix_to_str(matrix):
    res = ''
    for row in np.asarray(matrix):
        res+='    '
        for column in row:
            res+= f' {column}'
        res+='\n'
    return res
        
sigma = 1e-2
lattice = np.matrix([[0, 0.5, 0.5], [0.5, 0, 0.5], [0.5, 0.5, 0]])
deformation_matrix_zero = np.matrix([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
deformation_matrix_negative = np.matrix([[1, 0, 0], [-1e-2, 1, 0], [0, 0, 1]])
deformation_matrix_positive = np.matrix([[1, 0, 0], [1e-2, 1, 0], [0, 0, 1]])

In [16]:
lattice_config = """&control
    prefix = '{0}',
    pseudo_dir = '{1}',
    outdir = '{2}',
    calculation = 'relax',
    etot_conv_thr = 1.d-5,
    forc_conv_thr = 1.d-5,
/
&system
! better lattice parametrization
    ibrav = 0,
    celldm(1) = {3},
    ntyp = 1,
    nat = 2,
    ecutwfc = 100.0,
/
&electrons
/
&ions
ion_dynamics = 'bfgs',
/
ATOMIC_SPECIES
    X {4} {5}
ATOMIC_POSITIONS alat
    X 0.00 0.00 0.00
    X 0.25 0.25 0.25
K_POINTS automatic
    6 6 6 1 1 1
CELL_PARAMETERS alat
{6}"""

In [17]:
def calc_energy(celldim, matrix):
    conf = lattice_config.format(f'lattice_{celldim}',
                                pseudo_dir,
                                output_dir,
                                celldim,
                                atom.get_atomic_mass(),
                                atom.get_pseudo_file(),
                                matrix_to_str(matrix))
    p = Popen([pwx_exe], stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    stdout, _ = p.communicate(input=conf.encode())
    output = stdout.decode()
    return parse_output(output)

In [18]:
undeformed_celldim_energy = calc_energy(atom.get_celldim(), np.dot(lattice, deformation_matrix_zero))
compression_celldim_energy = calc_energy(atom.get_celldim(), np.dot(lattice, deformation_matrix_negative))
stretching_celldim_energy = calc_energy(atom.get_celldim(), np.dot(lattice, deformation_matrix_positive))

In [28]:
d2E = numerical_derivative(undeformed_celldim_energy, compression_celldim_energy, stretching_celldim_energy,sigma)
C44 = calc_C44(gamma, d2E)

print(f'C44 = {C44}')

C44 = 0.0788016825584805


### 3. Расчет констант эластичности С11 и C12

In [22]:
deformation_matrix_c11_negative = np.matrix([[1 - sigma, 0, 0], [0, 1 - sigma, 0], [0, 0, 1]])
deformation_matrix_c11_positive = np.matrix([[1 + sigma, 0, 0], [0, 1 + sigma, 0], [0, 0, 1]])

deformation_matrix_c12_negative = np.matrix([[1 - sigma, 0, 0], [0, 1 + sigma, 0], [0, 0, 1]])
deformation_matrix_c12_positive = np.matrix([[1 + sigma, 0, 0], [0, 1 - sigma, 0], [0, 0, 1]])

In [23]:
compression_celldim_c11_energy = calc_energy(atom.get_celldim(), np.dot(lattice, deformation_matrix_c11_negative))
stretching_celldim_c11_energy = calc_energy(atom.get_celldim(), np.dot(lattice, deformation_matrix_c11_positive))

compression_celldim_c12_energy = calc_energy(atom.get_celldim(), np.dot(lattice, deformation_matrix_c12_negative))
stretching_celldim_c12_energy = calc_energy(atom.get_celldim(), np.dot(lattice, deformation_matrix_c12_positive))

In [27]:
d2Ec11 = numerical_derivative(undeformed_celldim_energy, compression_celldim_c11_energy, stretching_celldim_c11_energy,sigma)
d2Ec12 = numerical_derivative(undeformed_celldim_energy, compression_celldim_c12_energy, stretching_celldim_c12_energy,sigma)

In [26]:
C11 = calc_C11(gamma, d2Ec11, d2Ec12)
C12 = calc_C12(gamma, d2Ec11, d2Ec12)

print(f'C11 = {C11}')
print(f'C12 = {C12}')

C11 = 0.09525168303290511
C12 = 0.02248870622449062
